## `Generating and filling the keywords column to test the llama integration`

In [1]:
! pip install mysql-connector-python
! pip install requests
! pip install groq

In [1]:
import mysql.connector
import requests

In [2]:
# Extract data from MySQL database
try:
    conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="project_db"
    )
    cursor = conn.cursor()
    print("Connected to MySQL")
    cursor.execute("SELECT review_id,text FROM reviews")
    data = cursor.fetchall()

except mysql.connector.Error as e:
    print(e)


Connected to MySQL


In [3]:
data

[(1, 'perfect'),
 (2,
  "So far extremely happy with the iPhone 6.  This works perfectly for what it was purchased for and nice to have the larger storage.  Wasn't sure what iPhone to get, but very happy with this purchase and would highly recommend to anyone looking to get one.  While it isn't the latest model, it still has many of the extra added features advertised by Apple.  no regrets and happy to have this one"),
 (3,
  'I would not recommend buying any electronics from this company; my phone came defected.'),
 (4,
  'Battery is trashed essentially making the phone unusable.  Complete waste of money.'),
 (5,
  'This is literally a brand new phone for a used phone price. I could not find a single scratch on this phone. I am so happy with this purchase. Saved a ton of money too.'),
 (6, 'My daughter loves this phone and has had no problems with it.'),
 (7, 'Works well!'),
 (8,
  'This iPhone 8+ was no good out the box.  Screen Res was horrible, volume no good.  Will be returning it

In [4]:
reviews = []
for i in range(5):
    reviews.append(data[i])

reviews

[(1, 'perfect'),
 (2,
  "So far extremely happy with the iPhone 6.  This works perfectly for what it was purchased for and nice to have the larger storage.  Wasn't sure what iPhone to get, but very happy with this purchase and would highly recommend to anyone looking to get one.  While it isn't the latest model, it still has many of the extra added features advertised by Apple.  no regrets and happy to have this one"),
 (3,
  'I would not recommend buying any electronics from this company; my phone came defected.'),
 (4,
  'Battery is trashed essentially making the phone unusable.  Complete waste of money.'),
 (5,
  'This is literally a brand new phone for a used phone price. I could not find a single scratch on this phone. I am so happy with this purchase. Saved a ton of money too.')]

In [5]:
# Separate into two lists
review_ids, texts = zip(*reviews)

# Convert the zip object into lists
review_ids = list(review_ids)
texts = list(texts)

print(review_ids)  # Output: [1, 2, 3, 4]
print(texts)  # Output: ['a', 'b', 'c', 'd']


[1, 2, 3, 4, 5]
['perfect', "So far extremely happy with the iPhone 6.  This works perfectly for what it was purchased for and nice to have the larger storage.  Wasn't sure what iPhone to get, but very happy with this purchase and would highly recommend to anyone looking to get one.  While it isn't the latest model, it still has many of the extra added features advertised by Apple.  no regrets and happy to have this one", 'I would not recommend buying any electronics from this company; my phone came defected.', 'Battery is trashed essentially making the phone unusable.  Complete waste of money.', 'This is literally a brand new phone for a used phone price. I could not find a single scratch on this phone. I am so happy with this purchase. Saved a ton of money too.']


In [29]:
import os
from groq import Groq

GROQ_API_TOKEN = "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"
os.environ["GROQ_API_KEY"] = GROQ_API_TOKEN

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


def llama3_70b(prompt, temperature=0.0, input_print=True):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="llama3-70b-8192",
      temperature=temperature,
  )

  return (chat_completion.choices[0].message.content)

In [30]:
prompt ="""Recognize all aspect terms with their corresponding sentiment polarity in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. Determine the sentiment polarity from the options [\"positive\", \"negative\", \"neutral\"]. Answer in the format [\"aspect\", \"sentiment\"] without any explanation. If no aspect term exists, then only answer \"[]\"."
          """

In [31]:
aspects = []
output = []
for review in reviews:
  prompt_text = prompt+"'''"+review[1]+"'''"
  result = llama3_70b(prompt_text)
  aspects.append(result)

In [32]:
aspects

['[["perfect", "positive"]]',
 '[["iPhone 6", "positive"], ["storage", "positive"], ["purchase", "positive"], ["features", "positive"]]',
 '["electronics", "negative"], ["phone", "negative"], ["company", "negative"]',
 '["Battery", "negative"], ["phone", "negative"]',
 '[["phone", "positive"], ["price", "positive"], ["purchase", "positive"]]']

In [33]:
review_id = []
for review in reviews:
  review_id.append(review[0])

In [34]:
# Update each record based on review ID
for review_id, aspect in zip(review_id, aspects):
        cursor.execute(
            "UPDATE reviews SET keywords = %s WHERE review_id = %s",
            (aspect, review_id)
        )

conn.commit()
print("Data updated successfully.")

Data updated successfully.


# `Generating Predictions`

In [44]:
import mysql.connector
import json

try:
    conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="project_db"
    )
    cursor = conn.cursor()
    print("Connected to MySQL")
    cursor.execute("SELECT review_id,text FROM reviews")
    data = cursor.fetchall()

except mysql.connector.Error as e:
    print(e)

# Define the parent_asin to query
parent_asin = 'B07CQNF813'  # Replace with actual ASIN

# Query to select product specifications and keywords
query = """
SELECT keywords
FROM reviews
WHERE parent_asin = %s
"""

cursor.execute(query, (parent_asin,))
results = cursor.fetchall()

Connected to MySQL


In [45]:
results

[('["Battery", "negative"], ["phone", "negative"]',),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,)]

In [46]:
# Extract product specifications and keywords
product_specifications = None
negative_keywords = []

# Convert the tuple elements to lists
keywords_list = []
for item in results:
    if item[0] is not None:
        lists = eval(f'[{item[0]}]')
        keywords_list.extend(lists)  # Extend the main list with these sublists
print(keywords_list)
    
# Filter for negative keywords
for keyword in keywords_list:
    if keyword[1] == 'negative':
        negative_keywords.append(keyword[0])


#print("Product Specifications:", product_specifications)
print("Negative Keywords:", negative_keywords)

[['Battery', 'negative'], ['phone', 'negative']]
Negative Keywords: ['Battery', 'phone']


In [47]:
# Define the parent_asin to query
parent_asin = 'B07CQNF813'  # Replace with actual ASIN

# Query to select product specifications and keywords
query = """
SELECT features
FROM products
WHERE parent_asin = %s
"""

cursor.execute(query, (parent_asin,))
features = cursor.fetchall()

In [48]:
features

[("'Locked to the GSM network and only compatible with GSM carriers like AT&T, T-Mobile and Metro. Will not work with Sprint, Verizon, US Cellular or Cricket.', 'The device does not come with headphones or a SIM card. It does include a charger and charging cable that may be generic, in which case it will be UL or Mfi (Made for iPhone) Certified.', 'Inspected and guaranteed to have minimal cosmetic damage, which is not noticeable when the device is held at arms length.', 'Successfully passed a full diagnostic test which ensures like-new functionality and removal of any prior-user personal information.', 'Tested for battery health and guaranteed to have a minimum battery capacity of 80%.'",)]

In [49]:
import os
from groq import Groq

GROQ_API_TOKEN = api_token
os.environ["GROQ_API_KEY"] = GROQ_API_TOKEN

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


def llama3_70b(prompt, temperature=0.0, input_print=True):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="llama3-70b-8192",
      temperature=temperature,
  )

  return (chat_completion.choices[0].message.content)

In [17]:
#prompt = """Given the negative aspects of a product and its specifications, generate improvement suggestions in the form of actionable statements. The negative aspects are phrases or terms that indicate issues or dissatisfaction with the product, while the product specifications detail the product's features and characteristics. Use the following format for the input:

#Negative Aspects: [list of negative aspects]
#Product Specifications: [details of the product]

#Provide suggestions for improving the product based on the given information, focusing on addressing the negative aspects while considering the product specifications. Present each suggestion as a concise and actionable statement."""

In [50]:
prompt = """Given the negative aspects of a product and its specifications, generate improvement suggestions in the form of actionable statements. The negative aspects are phrases or terms that indicate issues or dissatisfaction with the product, while the product specifications detail the product's features and characteristics.

Input Format:

Negative Aspects: [list of negative aspects]
Product Specifications: [details of the product]
Output Format:

Negative Aspects: [list of negative aspects]
Improvement Suggestions: [list of actionable suggestions]
Provide improvement suggestions that directly address the negative aspects, while considering the product specifications. Present each suggestion as a concise and actionable statement."""

In [51]:
result = prompt+"'''"+"Negative Aspects: "+str(negative_keywords)+" Product Specifications: "+str(features)+"'''"
suggestions = llama3_70b(result)

In [21]:
suggestions

"Here is the output:\n\nNegative Aspects: ['Battery', 'phone']\n\nImprovement Suggestions:\n\n* Consider increasing the minimum battery capacity guarantee to 90% to address battery life concerns.\n* Expand compatibility to include CDMA networks, such as Sprint and Verizon, to cater to a broader range of customers.\n* Provide a higher-quality, brand-specific charger and cable to improve the overall user experience.\n* Offer a bundled package that includes headphones and a SIM card to enhance the value proposition for customers.\n* Implement a more rigorous inspection process to ensure that cosmetic damage is virtually unnoticeable, even when the device is held at close range.\n\nThese improvement suggestions directly address the negative aspects mentioned, while taking into account the product specifications. Each suggestion is concise, actionable, and focused on enhancing the product's features and user experience."

In [52]:
print(suggestions)

Here is the output:

Negative Aspects: ['Battery', 'phone']

Improvement Suggestions:

* Consider increasing the minimum battery capacity guarantee to 90% to address battery life concerns.
* Expand compatibility to include CDMA networks, such as Sprint and Verizon, to cater to a broader range of customers.
* Provide a higher-quality, brand-specific charger and cable to improve overall user experience.
* Offer a bundled package that includes headphones and a SIM card to enhance the overall value proposition.
* Implement a more rigorous inspection process to ensure that cosmetic damage is virtually unnoticeable, even when the device is held at close range.

These improvement suggestions directly address the negative aspects mentioned, while taking into account the product specifications. Each suggestion is presented as a concise and actionable statement.


In [53]:
def format_suggestions(input_text):
    lines = input_text.split('\n')
    formatted_output = []

    in_suggestions_section = False

    for line in lines:
        if line.startswith("**Negative Aspects:**"):
            formatted_output.append("Improvement Suggestions\n")
            formatted_output.append("Negative Aspects Identified:\n")
            in_suggestions_section = False

        elif line.startswith("* "):
            # Add bullet point for negative aspects and specifications
            formatted_output.append(f"- {line[2:]}\n")
        
        elif line.startswith("Product Specifications:"):
            formatted_output.append("\nProduct Specifications:\n")
            in_suggestions_section = False
        
        elif line.startswith("1.") or line.startswith("2.") or line.startswith("3.") or line.startswith("4.") or line.startswith("5."):
            if not in_suggestions_section:
                formatted_output.append("\nSuggestions for Improvement:\n")
                in_suggestions_section = True
            # Add numbered improvement suggestions
            formatted_output.append(f"{line}\n")
        
        else:
            # Keep any other lines as they are
            formatted_output.append(line.strip() + "\n")

    return ''.join(formatted_output)

# Example usage
input_text = suggestions

formatted_output = format_suggestions(input_text)
print(formatted_output)


Here is the output:

Negative Aspects: ['Battery', 'phone']

Improvement Suggestions:

- Consider increasing the minimum battery capacity guarantee to 90% to address battery life concerns.
- Expand compatibility to include CDMA networks, such as Sprint and Verizon, to cater to a broader range of customers.
- Provide a higher-quality, brand-specific charger and cable to improve overall user experience.
- Offer a bundled package that includes headphones and a SIM card to enhance the overall value proposition.
- Implement a more rigorous inspection process to ensure that cosmetic damage is virtually unnoticeable, even when the device is held at close range.

These improvement suggestions directly address the negative aspects mentioned, while taking into account the product specifications. Each suggestion is presented as a concise and actionable statement.



In [56]:
# Prepare the SQL query to insert data
insert_query = """
UPDATE products SET improvements = %s WHERE parent_asin = %s
"""

def update_improvements(parent_asin, formatted_output):
        try:
            cursor.execute(
                insert_query,
                (formatted_output, parent_asin)
            )
            conn.commit()
            print("Data updated successfully.")
        except Exception as e:
            print("Error updating data:", str(e))

In [57]:
update_improvements(parent_asin, formatted_output)

Data updated successfully.
